# Modeling Approaches

In [1]:
import pandas as pd
import numpy as np
# import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict
# Week 7
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import accuracy_score
# import xgboost as xgb

# Week 8
from sklearn.ensemble import RandomForestClassifier

## Reading into the data

In [2]:
pd.set_option('display.max_columns', None)

train = pd.read_csv('train_final.csv', low_memory=False)
validation = pd.read_csv('val_set_final.csv')

In [3]:
train.head()

,Unnamed: 0,date,customer_code,employee_index,country,female,age,new_cust,seniority_in_months,cust_type,residency_spain,birth_spain,join_channel,province_name,active_cust,income,segment,savings_acct,guarantees,current_acct,derivada_acct,payroll_acct,junior_acct,mas_particular_acct,particular_acct,particular_plus_acct,short_term_depo,medium_term_depo,long_term_depo,e_acct,funds,mortgage,pension,loans,taxes,credit_card,securities,home_acct,pensions_2,direct_debt,total_products,01 - TOP,02 - PARTICULARES,03 - UNIVERSITARIO,join_channel_encoded,province_name_encoded,employee_index_encoded,customer_code_encoded,income_to_age
0,0,2015-07-28,664160,N,1,0,0.632653,0,0.402344,1,1,0,KAR,MADRID,0,1.989686,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.424185,1.749698,1.407278,263662,3.144938
1,1,2016-01-28,1076784,N,1,0,0.214286,0,0.152344,1,1,0,KHE,LERIDA,0,-0.306603,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.886876,1.006139,1.407278,459750,-1.430748
2,2,2015-12-28,672465,N,1,0,0.387755,0,0.417969,1,1,0,KFC,SEVILLA,1,-0.148205,02 - PARTICULARES,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,0,1,0,1.559984,1.382030,1.407278,266890,-0.382203
3,3,2015-10-28,774528,N,1,0,0.397959,0,0.343750,1,1,0,KFA,MURCIA,1,-0.228531,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,1,0,1.850124,1.075147,1.407278,300528,-0.574243
4,4,2016-05-28,569598,N,1,0,0.459184,0,0.496094,1,1,0,KAT,MADRID,1,0.588748,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.942077,1.749698,1.407278,227731,1.282133


In [4]:
validation.head()

,Unnamed: 0.1,Unnamed: 0,date,customer_code,employee_index,country_spain,female,age,first_contract_date,new_cust,seniority_in_months,primary_cust,last_date_primary,cust_type,residency_spain,birth_spain,join_channel,deceased,province_name,active_cust,income,segment,savings_acct,guarantees,current_acct,derivada_acct,payroll_acct,junior_acct,mas_particular_acct,particular_acct,particular_plus_acct,short_term_depo,medium_term_depo,long_term_depo,e_acct,funds,mortgage,pension,loans,taxes,credit_card,securities,home_acct,payroll_acct.1,pensions_2,direct_debt,total_products,01 - TOP,02 - PARTICULARES,03 - UNIVERSITARIO,join_channel_encoded,province_name_encoded,employee_index_encoded,income_to_age
0,0,0,2015-11-28,161428,N,1,1,0.744898,2000-03-16,0,0.738281,1,0,1,1,1,KAS,0,MADRID,0,-0.001810,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.015705,1.749698,1.407278,-0.002430
1,1,1,2015-12-28,367478,N,1,1,0.418367,2002-07-02,0,0.632812,1,0,1,1,0,KAT,0,MADRID,1,1.473140,02 - PARTICULARES,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,5,0,1,0,1.942077,1.749698,1.407278,3.521079
2,2,2,2015-11-28,643150,N,1,0,0.520408,2006-09-21,0,0.429688,1,0,1,1,0,KFC,0,ALBACETE,0,-0.131613,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.559984,1.103044,1.407278,-0.252899
3,3,3,2016-04-28,1385854,N,1,0,0.367347,2015-03-03,0,0.050781,1,0,1,1,0,KHK,0,SEVILLA,0,-0.056218,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.122151,1.382030,1.407278,-0.153034
4,4,4,2015-08-28,495733,N,1,0,0.346939,2004-10-13,0,0.507812,1,0,1,1,0,KFC,0,MADRID,0,0.289349,02 - PARTICULARES,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1.559984,1.749698,1.407278,0.833982


## Pre-processing  
Changing columns name and dropping columns so both datasets are the same

In [5]:
train = train.rename(columns={'country': 'country_spain'})

In [6]:
train = train.drop(columns=['Unnamed: 0'])
validation = validation.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
drop = ['join_channel', 'province_name', 'employee_index', 'segment', 'total_products']
train = train.drop(columns=drop + ['customer_code_encoded'])
validation = validation.drop(columns=drop + ['payroll_acct.1', 'first_contract_date', 'primary_cust', 'last_date_primary', 'deceased'])

Setting products we want to predict

In [7]:
products = ['savings_acct', 'guarantees', 'current_acct',
       'derivada_acct', 'payroll_acct', 'junior_acct', 'mas_particular_acct',
       'particular_acct', 'particular_plus_acct', 'short_term_depo',
       'medium_term_depo', 'long_term_depo', 'e_acct', 'funds', 'mortgage',
       'pension', 'loans', 'taxes', 'credit_card', 'securities', 'home_acct',
       'pensions_2', 'direct_debt']

Dropping duplicates on customer code column since the last instance will show all the products a client has 

In [8]:
train = train.drop_duplicates(subset=['customer_code'], keep='last')
validation = validation.drop_duplicates(subset=['customer_code'], keep='last')

# Removing customers from validation set that appear in training set
validation = validation[~validation['customer_code'].isin(train['customer_code'])]

In [9]:
print(train.info())
print(validation.info())

<class 'pandas.core.frame.DataFrame'>
Index: 706866 entries, 39288 to 6579716
Data columns (total 42 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date                    706866 non-null  object 
 1   customer_code           706866 non-null  int64  
 2   country_spain           706866 non-null  int64  
 3   female                  706866 non-null  int64  
 4   age                     706866 non-null  float64
 5   new_cust                706866 non-null  int64  
 6   seniority_in_months     706866 non-null  float64
 7   cust_type               706866 non-null  int64  
 8   residency_spain         706866 non-null  int64  
 9   birth_spain             706866 non-null  int64  
 10  active_cust             706866 non-null  int64  
 11  income                  706866 non-null  float64
 12  savings_acct            706866 non-null  int64  
 13  guarantees              706866 non-null  int64  
 14  current_acct        

Defining our Xs and Ys

In [10]:
X_train = train.drop(['customer_code', 'date'] + products, axis=1)
y_train = train[products]

X_val = validation.drop(['customer_code', 'date'] + products, axis=1)
y_val = validation[products]

In [11]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (706866, 17)
Shape of y_train: (706866, 23)
Shape of X_val: (200333, 17)
Shape of y_val: (200333, 23)


# Week 8 - Develop Third Modeling Approach

Setting hyperparameters

In [12]:
hyperparameter_variations = [
    {'n_estimators': 50, 'max_depth': 10, 'min_samples_leaf': 5, 'random_state':17},
    {'n_estimators': 100, 'max_depth': 15, 'min_samples_leaf': 3, 'random_state':17},
    {'n_estimators': 150, 'max_depth': 20, 'min_samples_leaf': 2, 'random_state':17}
]

Creating funtion to train and evaluate the Random Forest model  
  
This week we will keep the same metrics used last week -ROC AUC, F1 Score and Confusion Matrix, so we can compare in the end which model is better  

In [13]:
def train_and_evaluate_rf(X_train, y_train, X_val, y_val, params):
    metrics = {}
    for product in y_train.columns:
        # Train the model
        model = RandomForestClassifier(**params, n_jobs=-1)
        model.fit(X_train, y_train[product])
        
        # Make predictions
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)
        
        # Calculate metrics
        metrics[product] = {
            'train': calculate_metrics(y_train[product], y_train_pred),
            'val': calculate_metrics(y_val[product], y_val_pred)
        }
    
    return metrics

def calculate_metrics(y_true, y_pred):
    return {
        'ROC AUC': roc_auc_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred),
        'Accuracy': accuracy_score(y_true, y_pred),
        'Confusion Matrix': confusion_matrix(y_true, y_pred)
    }

In [ ]:
# Train and evaluate models for each variation
results = []
for i, params in enumerate(hyperparameter_variations):
    metrics = train_and_evaluate_rf(X_train, y_train, X_val, y_val, params)
    
    avg_train_roc_auc = np.mean([metrics[product]['train']['ROC AUC'] for product in y_train.columns])
    avg_train_f1 = np.mean([metrics[product]['train']['F1 Score'] for product in y_train.columns])
    avg_train_accuracy = np.mean([metrics[product]['train']['Accuracy'] for product in y_train.columns])
    
    avg_val_roc_auc = np.mean([metrics[product]['val']['ROC AUC'] for product in y_train.columns])
    avg_val_f1 = np.mean([metrics[product]['val']['F1 Score'] for product in y_train.columns])
    avg_val_accuracy = np.mean([metrics[product]['val']['Accuracy'] for product in y_train.columns])
    
    results.append({
        'Variation': f'Variation {i+1}',
        'Train ROC AUC': avg_train_roc_auc,
        'Train F1 Score': avg_train_f1,
        'Train Accuracy': avg_train_accuracy,
        'Val ROC AUC': avg_val_roc_auc,
        'Val F1 Score': avg_val_f1,
        'Val Accuracy': avg_val_accuracy
    })

Creating a table to summarize results and define which variation is the best

In [35]:
results_df = pd.DataFrame(results)
print("Results Table:")
print(results_df.to_string(index=False))

best_model_idx = results_df['Val ROC AUC'].idxmax()
best_model = results_df.loc[best_model_idx]['Variation']
print(f"\n The best model for this week is: {best_model}")

Results Table:
  Variation  Train ROC AUC  Train F1 Score  Train Accuracy  Val ROC AUC  Val F1 Score  Val Accuracy
Variation 1       0.550031        0.138405        0.959554     0.541662      0.112408      0.971892
Variation 2       0.565750        0.184944        0.961033     0.550482      0.138092      0.971856
Variation 3       0.591426        0.260752        0.964707     0.554772      0.152463      0.971651

 The best model for this week is: Variation 3


Creating a table to see the results in a easier to interpret way

In [26]:
# Print detailed metrics for each product
for product in y_train.columns:
    print(f"Product: {product}")
    print(f"Train - ROC AUC: {metrics[product]['train']['ROC AUC']:.4f}, F1 Score: {metrics[product]['train']['F1 Score']:.4f}")
    print(f"Val - ROC AUC: {metrics[product]['val']['ROC AUC']:.4f}, F1 Score: {metrics[product]['val']['F1 Score']:.4f}")
    print(f"Confusion Matrix (Validation):\n{metrics[product]['val']['Confusion Matrix']}\n")

Product: savings_acct
Train - ROC AUC: 0.5000, F1 Score: 0.0000
Val - ROC AUC: 0.5000, F1 Score: 0.0000
Confusion Matrix (Validation):
[[200325      0]
 [     8      0]]

Product: guarantees
Train - ROC AUC: 0.5000, F1 Score: 0.0000
Val - ROC AUC: 0.5000, F1 Score: 0.0000
Confusion Matrix (Validation):
[[200331      0]
 [     2      0]]

Product: current_acct
Train - ROC AUC: 0.7536, F1 Score: 0.8195
Val - ROC AUC: 0.7306, F1 Score: 0.8072
Confusion Matrix (Validation):
[[ 48475  29677]
 [ 19423 102758]]

Product: derivada_acct
Train - ROC AUC: 0.5000, F1 Score: 0.0000
Val - ROC AUC: 0.5000, F1 Score: 0.0000
Confusion Matrix (Validation):
[[200289      0]
 [    44      0]]

Product: payroll_acct
Train - ROC AUC: 0.5469, F1 Score: 0.1710
Val - ROC AUC: 0.5132, F1 Score: 0.0517
Confusion Matrix (Validation):
[[193931    188]
 [  6044    170]]

Product: junior_acct
Train - ROC AUC: 0.9982, F1 Score: 0.9885
Val - ROC AUC: 0.9703, F1 Score: 0.9331
Confusion Matrix (Validation):
[[199129    

## Generate product recommendations  
Here we want to visualize the product recommendations for each customer  
We will generate the product recommendations using Variation 3 - that had the best performance between all variations

In [21]:
best_variation = 'Variation 3'
best_params = hyperparameter_variations[int(best_variation.split()[-1]) - 1]

# Train models and generate predictions
product_models = {}
for product in y_train.columns:
    clf = RandomForestClassifier(**best_params, n_jobs=-1)
    clf.fit(X_train, y_train[product])
    product_models[product] = clf

train_preds = {}
for product in y_train.columns:
    proba = product_models[product].predict_proba(X_train)[:, 1]

    for customer_id, prob in zip(train['customer_code'], proba):
        if customer_id not in train_preds:
            train_preds[customer_id] = []
        train_preds[customer_id].append((product, prob))


We want to make sure that we are not recommending a product that the customer already own, so we will store the products that customers already have

In [22]:
def get_active_products(customer_data):
    return set(product for product in y_train.columns if customer_data[product] > 0)

We will sort the recommended products by the probability of a client getting it and we will get the top 7 recommendations

In [23]:
# Sort and filter recommendations for each customer
for customer_id in train_preds:
    # Sort by probability
    sorted_prods = sorted(train_preds[customer_id], key=lambda x: x[1], reverse=True)
    customer_data = train[train['customer_code'] == customer_id].iloc[0]
    
    # Filter out products already owned
    active_products = get_active_products(customer_data)
    recommended_products = [prod for prod, _ in sorted_prods if prod not in active_products]

    # Get top 7
    train_preds[customer_id] = recommended_products[:7]

print("Example Recommendations:")
for customer_id in list(train_preds.keys())[:5]:  # First 5 customers
    print(f"Customer {customer_id}: {train_preds[customer_id]}")

Example Recommendations:
Customer 1225385: ['current_acct', 'direct_debt', 'pensions_2', 'taxes', 'credit_card', 'payroll_acct', 'long_term_depo']
Customer 1358829: ['direct_debt', 'pensions_2', 'payroll_acct', 'taxes', 'e_acct', 'credit_card', 'long_term_depo']
Customer 1436539: ['direct_debt', 'pensions_2', 'payroll_acct', 'securities', 'short_term_depo', 'e_acct', 'long_term_depo']
Customer 1448049: ['current_acct', 'taxes', 'pensions_2', 'payroll_acct', 'junior_acct', 'mas_particular_acct', 'securities']
Customer 1396837: ['pensions_2', 'direct_debt', 'payroll_acct', 'e_acct', 'mas_particular_acct', 'credit_card', 'junior_acct']


Lastly, we want to identify which product has been recommended the most and least in the model

In [24]:
product_recommendation_counts = {product: sum(1 for recs in train_preds.values() if product in recs) for product in y_train.columns}
most_recommended = max(product_recommendation_counts, key=product_recommendation_counts.get)
least_recommended = min(product_recommendation_counts, key=product_recommendation_counts.get)

print(f"Most frequently recommended product: {most_recommended} ({product_recommendation_counts[most_recommended]} times)")
print(f"Least frequently recommended product: {least_recommended} ({product_recommendation_counts[least_recommended]} times)")

Most frequently recommended product: taxes (623341 times)
Least frequently recommended product: guarantees (1401 times)
